# Full model code

This notebook contains the **final** model code and docstrings generated by Perplexity.AI between June and end of July 2024.

The code is stored in a module called `s2_stroke_rehab_model.py`. It has been autoformatted using the tool `black`. This was used to aid PEP8 compliance of the code and help with line wrapping.   The module level docstring was added manually by the modeller (copied and modified from stage 1) and is included for archival purposes.

In [1]:
from IPython.display import Code

## Code stored in `s2_stroke_rehab_model.py`

In [4]:
Code(filename='s2_stroke_rehab_model.py', language='python')

"""A SimPy-based simulation model for analyzing patient flow in stroke care units.
INTERNAL REPRODUCTION RUN - STAGE 2

This module simulates the patient flow through an Acute Stroke Unit (ASU) and a connected
Rehabilitation Unit to assist in capacity planning for acute and community stroke services.
It models patient arrivals, treatment durations, and transitions between units using
statistical distributions to provide insights into occupancy levels and potential delays.

Authors: LLM implemented by Perplexity.AI
Auto-formatted with `black` for PEP8 compliance.

Based on:
---------
Monks et al. A modelling tool for capacity planning in acute 
and community stroke services
https://link.springer.com/article/10.1186/s12913-016-1789-4

All functions and classes included in this module have been 
generated between Jun-Jul 2024 by the LLM implemented by Perplexity.AI 
https://www.perplexity.ai/

Auto-formatting:
----------------
After generation was complete We used `black` to autoformat the code
to PEP8 standards

Prompts used:
-----------
The prompts were taken from stage 1 model creation. Additional prompts
were used when needed i.e. if the original prompt did not provide the 
desired output in stage 2.
"""

import math
import statistics
import simpy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# lognormal function


def lognormal_to_normal(mean, std):
    """
    Convert lognormal moments to normal moments.

    Parameters:
    mean (float): Mean of the lognormal distribution.
    std (float): Standard deviation of the lognormal distribution.

    Returns:
    tuple: (mean, std) of the corresponding normal distribution.
    """
    normal_mean = np.log(mean**2 / np.sqrt(std**2 + mean**2))
    normal_std = np.sqrt(np.log(std**2 / mean**2 + 1))
    return normal_mean, normal_std


# Experiment class


class Experiment:
    def __init__(self, params=None, random_number_set=0):
        default_params = {
            "results_collection_period": 5 * 365,
            "warm_up": 1095,
            "trace": False,
            "acute_audit_interval": 1,
            "rehab_audit_interval": 1,
            "rehab_stroke_iat": 21.8,
            "rehab_neuro_iat": 31.7,
            "rehab_other_iat": 28.6,
            "rehab_stroke_esd_los_mean": 30.3,
            "rehab_stroke_esd_los_std": 23.1,
            "rehab_stroke_other_los_mean": 28.4,
            "rehab_stroke_other_los_std": 27.2,
            "rehab_complex_neuro_los_mean": 27.6,
            "rehab_complex_neuro_los_std": 28.4,
            "rehab_other_los_mean": 16.1,
            "rehab_other_los_std": 14.1,
            "rehab_stroke_post_destination_probs": [0.4, 0.6],
            "rehab_complex_neuro_post_destination_probs": [0.09, 0.91],
            "rehab_other_post_destination_probs": [0.12, 0.88],
            "patient_types": {
                "Stroke": {
                    "interarrival_time": 1.2,
                    "post_asu_probabilities": [0.24, 0.13, 0.63],
                    "los_params": {
                        "Rehab": (7.4, 8.6),
                        "ESD": (4.6, 4.8),
                        "Other": (7.0, 8.7),
                    },
                },
                "TIA": {
                    "interarrival_time": 9.3,
                    "post_asu_probabilities": [0.01, 0.01, 0.98],
                    "los_params": (1.8, 5.0),
                },
                "Complex Neurological": {
                    "interarrival_time": 3.6,
                    "post_asu_probabilities": [0.11, 0.05, 0.84],
                    "los_params": (4.0, 5.0),
                },
                "Other": {
                    "interarrival_time": 3.2,
                    "post_asu_probabilities": [0.05, 0.10, 0.85],
                    "los_params": (3.8, 5.2),
                },
            },
        }

        if params is None:
            self.params = default_params
        else:
            self.params = self.merge_params(default_para